In [12]:
def get_data(endpoint, params):
    
    URL = "https://v3.football.api-sports.io/"
    headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
    }
    
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data, remaining                                                                     

In [2]:
def encode_data(data_dict):
    encoded = []
    for key, val in data_dict.items():
        if isinstance(val, dict):
            encoded.extend(encode_data(val).items())
        elif isinstance(val, list):
            if val:
                if all(isinstance(i, dict) for i in val):
                    for k, v in enumerate(val):
                        encoded.extend(encode_data(v).items())
                else:
                    encoded.append((key, val))
            else:
                encoded.append((key, []))
        else:
            encoded.append((key, val))
            
    return dict(encoded)

In [3]:
def data_to_sql(table_name, df, db_params):

    try:
        # Establish the connection
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()
        
        #insert data into tables
        insert_query = """
            INSERT INTO {} ({})
            VALUES ({})
        """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)))
        pgcursor.executemany(insert_query, df.values.tolist())
        print(f'table {table_name} updated')
        
        # Commit the changes
        conn.commit()
        
    
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        # Close the cursor and connection
        cur.close()
        conn.close()

In [36]:
import time
import boto3
import pandas as pd
import io
import psycopg2
import numpy as np
import requests
import json

# get current and european seasons files from bucket
s3 = boto3.client('s3')

bucket_name = 'sport-storage'
object_key1 = 'current.csv'
object_key2 = 'european_seasons.csv'
object_key3 = 'teams.json'

response1 = s3.get_object(Bucket = bucket_name, Key = object_key1)
response2 = s3.get_object(Bucket = bucket_name, Key = object_key2)
response3 = s3.get_object(Bucket = bucket_name, Key = object_key3)

current = pd.read_csv(io.BytesIO(response1['Body'].read()))
european_seasons = pd.read_csv(io.BytesIO(response2['Body'].read()))

# insert parameters to be found
league = current['league_id'][0]
year = current['year'][0]
to_find = current['type'][0]

# get current index
index = np.where((european_seasons['league_id']==league)&(european_seasons['year']==year))[0][0]

remaining = 0
total_team_stats_data = []
total_teams_data = []
done = False

# if script ended on collecting team stats get rest of teams
if to_find == 'teams/statistics':
    teams_list = pickle.loads(response3['Body'].read())
else:
    teams_list = []

# if limit of requests wasnt reached and there is still something to collect enter loop
while (done==False)&(remaining > 0):

    # get data from API
    if to_find == 'teams': 
        params = {'league': league,
                  'season': year
                 }
    else:
        team = teams_list[0]
        params = {'league': league,
                  'season': year,
                  'team': team
                 }
        
    endpoint = to_find
    data, remaining_req = get_data(endpoint, params)
    
    # preprocess data
    if endpoint == 'teams':
        total_teams_data.extend(encode_data(team) for team in data['response'])
    else:
        total_team_stats_data.append(encode_data(data['response']))

    # if last endpoint is teams now find stats for every team in that season
    if endpoint == 'teams':
        to_find = 'teams/statistics'
        teams_list = [row['team']['id'] for row in data['response']]

    # if team stats are collecting drop team that data was already collected for
    else:
        (endpoint == 'teams/statistics') & (len(teams_list) > 0)
        teams_list.pop(0)

    # if team stats were collected for every team in current season move to next season
        if (endpoint == 'teams/statistics') & (len(teams_list) == 0) & (index < len(european_seasons)-1):
            index += 1
            league = european_seasons.loc[index]['league_id']
            year = european_seasons.loc[index]['year']
            to_find = 'teams'

    # if data for all seasons were collected quit loop
        elif (endpoint == 'teams/statistics') & (len(teams_list) == 0) & (index == len(european_seasons)-1):
            done = True
        
    remaining = int(remaining_req)
    print(remaining)
    # sleep cause there can be done only 10 requests per minute
    time.sleep(7)
    
    # saving to sql database
    # db parameters
db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}


if len(total_team_stats_data) > 0:
    team_stats_df = pd.DataFrame(total_team_stats_data)
    data_to_sql('team_stats', team_stats_df, db_params)
                
if len(total_teams_data) > 0:
    teams_df = pd.DataFrame(teams_df)
    data_to_sql('teams', teams_df, db_params)

if not done:
    #
    json_data = json.dumps(teams_list)
    s3.put_object(Bucket = bucket_name, Key = object_key3, Body=json_data)
    data = {'league_id': [league], 'year': [year], 'type': [to_find]}

    current = pd.DataFrame(data)
    s3.put_object(Bucket = bucket_name, Key = object_key1, Body=current)
    print(f'{len(european_seasons)-index-1} seasons left')
else:
    print('Teams and their statistics were collected for every season played in Europe')

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value:    league_id  year   type
0         39  2010  teams, type: <class 'pandas.core.frame.DataFrame'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object

[]